# 내장 디스크립터

* property
* classmethod
* staticmethod

## property 사용법

In [1]:
class A:
    def __init__(self, value):
        self.value = value
        
    @property
    def double(self):
        return self.value * 2

In [2]:
a = A(10)

In [3]:
a.value, a.double

(10, 20)

In [4]:
class OnlyInt:

    @property
    def data(self):
        return self._data
    
    @data.setter
    def data(self, value):
        if isinstance(value, int):
            self._data = value
        else:
            raise AttributeError("only int")
            
    @data.deleter
    def data(self):
        raise AttributeError("can't")

In [5]:
t = OnlyInt()

In [6]:
t.data = 12

In [7]:
t.data

12

In [8]:
try :
    del t.data
except Exception as e :
    print(e)

can't


In [9]:
try :
    t.data = '123'
except Exception as e :
    print(e)

only int


property는 표준디스크립터

get, set, delete 로직을 지정할 수 있는..

정보를 instance 저장하게하는..

# 도전 : 내장 property로 구현

### 인스턴스를 생성할때 반지름을 입력받는 원 객체 / 원의 속성을 가지고 있는..

> 반지름은 mutable    
> 그 외에 지름, 둘레, 넓이는 immutable



In [10]:
def no(*args):
    raise AttributeError("you can't")


class Circle:
    pi = 3.14
    
    def __init__(self, radius):
        self.radius = radius
    
    @property
    def diameter(self):
        return self.radius * 2

    @property
    def circumference(self):
        return self.radius * self.pi * 2
    
    @property
    def area(self):
        return self.radius ** 2 * self.pi

#     area = area.setter(no)
#     circumference = circumference.setter(no)

In [11]:
c = Circle(10)

In [12]:
try :
    c.area = 12
except Exception as e :
    print(e)

can't set attribute


# 내장 property 구현체

In [13]:
class my_property:
    def __init__(self, fget=None, fset=None, fdel=None):
        self.fget = fget
        self.fset = fset
        self.fdel = fdel
        
    def setter(self, fset):
        self.fset = fset
        return self 
    
    def deleter(self, fdel):
        self.fdel = fdel
        return self
    
    def getter(self, fget):
        self.fget = fget
        return self

    def __get__(self, instance, owner):
        return self.fget(instance)
    
    def __set__(self, instance, value):
        if not self.fset:
            raise AttributeError
        return self.fset(instance, value)
    
    def __delete__(self, instance):
        if not self.fdel:
            raise AttributeError
        return self.fdel(instance)

In [14]:
class OnlyInt:
    @my_property
    def data(self):
        return self._data
    
    @data.setter
    def data(self, value):
        if isinstance(value, int):
            self._data = value
        else:
            raise AttributeError("only int")
            
    @data.deleter
    def data(self):
        raise AttributeError("can't")

In [15]:
c = OnlyInt()

In [16]:
c.data = 12

In [17]:
c.data

12

In [18]:
try :
    c.data = '123'
except Exception as e :
    print(e)

only int


# 내장 디스크립터 2 : classmethod

In [19]:
class A:
    def m(arg):
        print(arg)

    @classmethod
    def cm(arg):
        print(arg)

In [20]:
a = A()

In [21]:
a.m()

In [22]:
a.cm()

<class '__main__.A'>


## 도전 : classmethod 구현

In [23]:
from functools import partial


class my_classmethod:
    def __init__(self, func):
        self.func = func
    
    def __get__(self, instance, owner):
        return partial(self.func, owner)

In [24]:
class A:
    def m(arg):
        print(arg)

    @my_classmethod
    def cm(arg):
        print(arg)

In [25]:
a = A()
a.m(), a.cm()

<class '__main__.A'>


(None, None)

# 내장 디스크립터 : staticmethod

In [26]:
class A:
    @staticmethod
    def sum(a, b, c):
        return a + b + c

In [27]:
a = A()
a.sum(1, 2, 3)

6

# 도전 : staticmethod 구현

In [28]:
class my_staticmethod:
    def __init__(self, func):
        self.func = func
        
    def __get__(self, instance, owner):
        return self.func

In [29]:
class A:
    @my_staticmethod
    def sum(a, b, c):
        return a + b + c

In [30]:
a = A()
a.sum(1, 2, 3)

6

# `__get__`, `__set__`, `__delete__`

### 3.6   하나 추가 + `__set_name__`

#### 3.6 이전

In [31]:
class A:
    def __init__(self, value, name):
        self.value = value
        self.name = name
        
    def __get__(self, instance, onwer):
        print('get', self.name)
        return self.value


class B:
    x = A(10, 'x')
    y = A(20, 'y')

In [32]:
b = B()

In [33]:
b.x

get x


10

In [34]:
b.y

get y


20

### 3.6 이후

In [35]:
class A:
    def __init__(self, value):
        self.value = value

    def __set_name__(self, instance, name):
        print("init", name)
        self.name = name
        
    def __get__(self, instance, onwer):
        print('get', self.name)
        return self.value


class B:
    x = A(10)
    y = A(20)

init x
init y


In [36]:
b = B()

In [37]:
b.x

get x


10

In [38]:
b.y

get y


20